# Alternate Discussion Grader
### Grades and gives feedback for Weekly Discussions

## Instructions
- Run the Nickname Processor
- On Blackboard Week 'w' Discussion, ctrl+a, ctrl+c, enter in LibreOffice Calc with ctrl+shift+v (UTF-8, string delimeter '), save into folder Documents//0BSData//yyyy_mm as DiscussionGraderw_CODE.csv (text csv, UTF-8, string delimeter ").
- On Blackboard Week 'w' Discussion, collect thread, ctrl+a, ctrl+c, enter in LibreOffice Calc with ctrl+shift+v (UTF=8, string delimeter '), save into folder Documents//0BSData//yyyy_mm as DiscussionGraderw.csv (text csv, UTF-8, string delimeter ").
- Enter the course "session" in 'yyyy_mm' format, "week" in 'w' format, and run this cell!

In [1]:
session = '2022_06'
week = '7'

thing = ['initial post', 'engagement in replies', 'participation in discussion', 'spelling/grammar score']
benevolence = ['good', 'bad']
# 
adjective = ['great', 'good', 'nice', 'amazing', 'awesome', 'excellent'] 
a_adjective = ['great', 'good', 'nice']
an_adjective = ['amazing', 'awesome', 'excellent']
start_emphasis = ['Really', 'Very']
emphasis = ['', '',  'really', 'very']
job = ['job', 'work', 'effort'] #, 'results'
# 
do = ['be sure to', 'make sure to', 'be sure you', 'make sure you', 'try to'] #, 'challenge yourself to'
goal = ['at least 150 words in your initial post', 'at least 3 replies', 'at least 3 days in discussion', 'less than 10 spelling/grammar errors']
result = ['to improve', 'to increase your score', 'to earn full points', 'to get full points', 'to receive full credit', 'to earn full credit']
# 
ending = ['.', '!']
# 
style_good = [[start_emphasis, adjective, thing, ending], 
         [start_emphasis, adjective, job, 'on your', thing, ending],
         ['You did a',emphasis, a_adjective, job, 'on your', thing, ending],
         ['You have a', emphasis, a_adjective, thing, ending],
         ['You did an', an_adjective, job, 'on your', thing, ending],
         ['You have an', an_adjective, thing, ending],
         ['Your', thing, 'is', emphasis, adjective, ending],
         ['You did', emphasis, adjective, 'on your', thing, ending]] # 
# 
style_bad = [[do, 'aim for', goal, '.'],
             [do, 'shoot for', goal, result, ending],
             ['Shoot for', goal, result, ending],
             ['Aim for', goal, ending],
             [result, ',', do, 'aim for', goal, ending],
             [result, ',', 'aim for', goal, ending]] #
# 
punctuation = [' ', ',', '.', '!']

#########################

import warnings
warnings.simplefilter('ignore')
import pandas as pd
import nltk # For Error_Count
nltk.download('brown', quiet=True) 
from textblob import TextBlob
import math # For a grading function
import random as rand


# General functions

def Disp(in_list):
    for t_item in in_list:
        print(t_item,'\n')
    return

def Remove(in_list, in_remove):
    while in_remove in in_list:
        in_list.remove(in_remove)
    return in_list

def no_rep(long_list):
    res = []
    [res.append(x) for x in long_list if x not in res]
    return res

def sum_str(in_list):
    return " ".join(in_list)


# Task-specific functions

def Error_Count_and_Display(TEXT):  
    l1 = TextBlob(TEXT)
    l1 = l1.split()
    l2 = TextBlob(TEXT)
    l2 = l2.correct()
    l2 = l2.split()
    bad = 0
    bad_list = []
    for i in range(0, len(l1)):
        if l1[i] != l2[i]:
            bad += 1
            bad_list.append(l1[i])
        else:
            pass
    return [bad,bad_list]

def Word_Count(in_str):
    return len(in_str.split(' '))

def Check_Prefix(in_str, in_pref):
    l = len(in_pref)
    return in_str[:l] == in_pref

def grade_func(x,n):
    return math.floor(min(max(0,(x**3)*0.2 + math.log(x+0.000001) + 13.75),20)/n)


# Main functions

def Thread_str_to_Name(in_str):
    return in_str[in_str.index("User's profile picture ")+23:in_str.index('Posted Date:')]

def Thread_str_to_Date(in_str):
    t_date = in_str[in_str.index("Date:")+5:in_str.index('Status:')].split(' ')
    t_month = t_date[0]
    t_day = t_date[1][:-1]
    t_year = t_date[2]
    return [t_month,t_day,t_year]

def Full_Name_to_Nickname(in_name):
    if in_name == ' Jonathan Doane  INSTRUCTOR MANAGER':
        return 'INSTRUCTOR'
    else:
        return nickname_list[full_name_list.index(in_name)]

def Info_to_Thread(in_info):
    return 'Thread:' +in_info[0]+ 'Post:' +in_info[1]+ 'Author:' +in_info[2]+ 'Posted Date:' +in_info[3]+ 'Status:' +in_info[4]

def initials_df_to_Info(in_df):
    initial_months = pd.to_datetime(in_df.iloc[:,0]).dt.month_name().tolist()
    initial_dates = in_df.iloc[:,0].to_list()
    for i in range(len(initial_dates)):
        initial_dates[i] = initial_months[i]+' '+str(int(initial_dates[i][3:5]))+', 20'+str(initial_dates[i][6:9])+str(int(initial_dates[i][9:11]))+initial_dates[i][11:]
    #
    out_info_list = []
    l = len('  Options Menu: Thread')
    for i in range(len(in_df.iloc[:,0])):
        out_info_list.append([in_df.iloc[i,1][:-l],
                              in_df.iloc[i,1][:-l],
                              in_df.iloc[i,2],
                              initial_dates[i],
                              in_df.iloc[i,3]])
    return out_info_list


def students_df_to_List(in_df):
    return in_df.loc[:,'Full Name'].tolist()

def posts_df_to_List(in_df):
    t_df = in_df.copy()
    t_index = t_df.iloc[:,0].tolist().index('Mark (Click to see options)')
    t_index += 1
    t_df = t_df.iloc[t_index:-4,:]
    in_posts = t_df.iloc[:,0].tolist()
    Remove(in_posts, 'Reply Quote Mark as Unread')
    clean_posts = []
    for string in in_posts:
        if Check_Prefix(string,'Thread:') == True:
            t_post = [string]
            clean_posts.append(t_post)
        else:
            t_post.append(string)
    for i in range(len(clean_posts)):
        clean_posts[i] = [clean_posts[i][0], sum_str(clean_posts[i][1:])]
    return clean_posts.copy()

def initials_df_to_List(in_df):
    t_df = in_df.copy()
    t_df = t_df[t_df[2].notna()]
    t_df = t_df.iloc[:,2:]
    out_initials = []
    for info in initials_df_to_Info(t_df):
        thread = Info_to_Thread(info)
        out_initials.append(Info_to_Thread(info))
    return out_initials


def Comment(in_thing, in_good):
    in_good = in_good.lower()
    out_comment = []
    if in_good == 'good':
        t_style_list = style_good
    else:
        t_style_list = style_bad
    r = rand.choice(range(len(t_style_list)))    
    t_style = t_style_list[r]
    for t_i in range(len(t_style)):
        t_s = t_style[t_i]
        if type(t_s) == str:
            out_comment.append(t_s)
        elif t_s == thing or t_s == goal:
            out_comment.append(t_s[in_thing])
        else:
            out_comment.append(rand.choice(t_s))
    out_comment = ' '.join(out_comment)
    for punc in punctuation:
        out_comment = out_comment.replace(' '+punc, punc)
    return out_comment.capitalize()

def students_df_to_Grades(in_df, in_print=True):
    t_s = len(students_df_to_List(in_df))
    # in_df['Grades'] = [ 0 for _ in range(t_s) ]
    out_grades = []
    for t_i in range(t_s):
        initial_word_count = in_df.iloc[t_i,10]
        number_replies = in_df.iloc[t_i,8]
        distinct_dates = in_df.iloc[t_i,9]
        initial_error_count = in_df.iloc[t_i,11]
        t_grades = [min(math.floor(initial_word_count*(20/150)+40),60),
                  grade_func(number_replies,1),
                  grade_func(distinct_dates,2),
                  max(math.floor(initial_error_count*(-0.05)+10),5)]
        out_grades.append(t_grades)
        comments = []
        # 
        if t_grades[0] >= 55:
            comments.append(Comment(0,'good'))
        else:
            comments.append(Comment(0,'bad'))
        # 
        if t_grades[1] == 20:
            comments.append(Comment(1,'good'))
        else:
            comments.append(Comment(1,'bad'))
        # 
        if t_grades[2] == 10:
            comments.append(Comment(2,'good'))
        else:
            comments.append(Comment(2,'bad'))
        # 
        if t_grades[3] > 6:
            comments.append(Comment(3,'good'))
        else:
            comments.append(Comment(3,'bad'))
        # 
        comments = '  '.join(comments)
        #
        t_grades.append(comments)
        t_grades.append(in_df.iloc[t_i,12])
        if in_print == True:
            full_name = in_df.iloc[t_i,5]
            print(str(t_i+1)+'.')
            print(full_name,'\n')
            Disp(t_grades)
            print('\n'*2)
        else:
            pass
    in_df['Grades'] = out_grades
    return in_df


students_df =  pd.read_csv('..//..//..//Documents//0BSData/' + session + '//Nicknames.csv', sep='\t')
students = students_df_to_List(students_df)
posts_df = pd.read_csv('..//..//..//Documents//0BSData/' + session + '//DiscussionGrader' +week+ '.csv', sep='\t', header=None)
posts = posts_df_to_List(posts_df)
initials_df = pd.read_csv('..//..//..//Documents//0BSData/' + session + '//DiscussionGrader' +week+ '_CODE.csv', sep='\t', header=None)
initials = initials_df_to_List(initials_df)
threads = [v[0] for v in posts]

s = len(students)
students_df['Initial Post'] = [ '' for _ in range(s) ]
students_df['Number of Replies'] = [ [] for _ in range(s) ]
students_df['Number of Dates']= [ [] for _ in range(s) ]
for i in range(len(posts)):
    thread = posts[i][0]
    name = Thread_str_to_Name(thread)
    if name == 'Jonathan Doane  INSTRUCTOR MANAGER':
        pass
    else:
        s_id = students.index(name)
        if thread in initials:
            students_df.iloc[s_id,7] = posts[i][1]
            students_df.iloc[s_id,9].append(Thread_str_to_Date(thread))
        else:
            students_df.iloc[s_id,8].append(Thread_str_to_Date(thread))
            students_df.iloc[s_id,9].append(Thread_str_to_Date(thread))
students_df['Initial Word Count'] = students_df['Initial Post'].copy()
students_df['Initial Error Count'] = students_df['Initial Post'].copy()
students_df['Initial Errors'] = students_df['Initial Post'].copy()
for i in range(len(students_df.iloc[:,0])):
    students_df.iloc[i,8] = len(students_df.iloc[i,8])
    students_df.iloc[i,9] = len(no_rep(students_df.iloc[i,9]))
    students_df.iloc[i,10] = Word_Count(students_df.iloc[i,10])
    students_df.iloc[i,11] = Error_Count_and_Display(students_df.iloc[i,11])[0]
    students_df.iloc[i,12] = ', '.join(Error_Count_and_Display(students_df.iloc[i,12])[1])

students_df_to_Grades(students_df)

1.
Kayla Ballance 

48 

13 

8 

9 

To earn full credit, be sure you aim for at least 150 words in your initial post!  To improve, aim for at least 3 replies.  Make sure to shoot for at least 3 days in discussion to improve!  You did really great on your spelling/grammar score. 

metric, measurements, cleaners. 




2.
Amy Barrett 

40 

0 

0 

10 

Make sure to aim for at least 150 words in your initial post.  Try to shoot for at least 3 replies to increase your score!  Be sure to aim for at least 3 days in discussion.  You have a very nice spelling/grammar score! 

 




3.
Sierra Cadenhead 

47 

0 

6 

9 

Make sure to aim for at least 150 words in your initial post.  Aim for at least 3 replies.  Aim for at least 3 days in discussion!  You did excellent on your spelling/grammar score. 

measurements, bins, measurements 




4.
Dominique Clyburn 

40 

0 

0 

10 

Make sure you aim for at least 150 words in your initial post.  To earn full credit, make sure you aim for at least

,Last Name,First Name,Username,Student ID,Nickname_PRE,Full Name,Nickname,Initial Post,Number of Replies,Number of Dates,Initial Word Count,Initial Error Count,Initial Errors,Grades
0,Ballance,Kayla,b00645789,B00645789,NaN,Kayla Ballance,Kayla,The U.S. actually uses the metric system more ...,1,2,64,3,"metric, measurements, cleaners.","[48, 13, 8, 9, To earn full credit, be sure yo..."
1,Barrett,Amy,b00648791,B00648791,NaN,Amy Barrett,Amy,,0,0,1,0,,"[40, 0, 0, 10, Make sure to aim for at least 1..."
2,Cadenhead,Sierra,b00648523,B00648523,NaN,Sierra Cadenhead,Sierra,I do you find myself using measurements in my ...,0,1,59,3,"measurements, bins, measurements","[47, 0, 6, 9, Make sure to aim for at least 15..."
3,Clyburn,Dominique,b00646864,B00646864,NaN,Dominique Clyburn,Dominique,,0,0,1,0,,"[40, 0, 0, 10, Make sure you aim for at least ..."
4,Cooper,Michelle,b00650699,B00650699,NaN,Michelle Cooper,Michelle,"Hello Everyone, We use the metric system every...",3,2,141,12,"We, metric, liquids., We, We, metric, cpt, dia...","[58, 20, 8, 9, You have a very good initial po..."
5,Fender,Rebecca,b00650234,B00650234,Becca,Rebecca Fender,Becca,In the US the metric system is used for lots o...,1,1,44,10,"US, metric, meters,, liters,, grams., billing,...","[45, 13, 6, 9, Make sure you shoot for at leas..."
6,Fleming,Elizabeth,b00650111,B00650111,NaN,Elizabeth Fleming,Elizabeth,I think we should use metric simply because we...,2,1,64,9,"metric, weirdos, We, liters, metric., We, metr...","[48, 16, 6, 9, Aim for at least 150 words in y..."
7,Grandberry,Ariel,b00649494,B00649494,NaN,Ariel Grandberry,Ariel,,0,0,1,0,,"[40, 0, 0, 10, Aim for at least 150 words in y..."
8,Hampshire,Nikie,b00650048,B00650048,NaN,Nikie Hampshire,Nikie,The only thing that i know about the metric is...,7,2,96,10,"metric, metrics., ethe, imperal, gose, belive,...","[52, 20, 8, 9, Shoot for at least 150 words in..."
9,Hayes,Taiylor,b00646689,B00646689,NaN,Taiylor Hayes,Taiylor,Something in the U.S measured using the metric...,0,1,102,8,"metric, So, An, metric, meters, shoild, metric...","[53, 0, 6, 9, Try to aim for at least 150 word..."
